In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

from loguru import logger

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_async import util
util.startLoop()

# fnos and lots from Zerodha

In [ ]:
from utils import zerodha_lots_expiries

In [ ]:
df = zerodha_lots_expiries()

In [ ]:
df

# Zerodha margin test

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://zerodha.com/margin-calculator/Equity/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/39.0.2171.95 Safari/537.36'
}


# request to zerodha
data = requests.get(url, headers=headers)


# converting the response to beautifulsoup element for scraping
soup = BeautifulSoup(data.text)


# storing the html data into list format
final_data = []


# iterating over html to get the stock name and margin values
for k in range(0, len(soup.findAll('td'))):
    if soup.findAll('td')[k].text.endswith(":EQ"):
        final_data.append([soup.findAll('td')[k].text.strip(),
            soup.findAll('td')[k+2].text.strip()]
        )


# converting the list to dataframe
df = pd.DataFrame(final_data, columns=["Stock Name", "Margin Value"])


In [ ]:


# saving the dataframe(data) into cvs format
df.to_csv("zerodha_margin_values.csv", index=False)

In [ ]:
# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
COLS = _vars.COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()
